<a href="https://colab.research.google.com/github/skj092/Computer_Vision_Lab/blob/main/Mask_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 175725, done.
remote: Counting objects: 100% (8088/8088), done.
remote: Compressing objects: 100% (572/572), done.
remote: Total 175725 (delta 7594), reused 7944 (delta 7499), pack-reused 167637
Receiving objects: 100% (175725/175725), 346.36 MiB | 31.53 MiB/s, done.


In [1]:
!pip install --upgrade albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 8.3 MB/s 
     |████████████████████████████████| 48.3 MB 103 kB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
!pip uninstall opencv-python-headless -y

Found existing installation: opencv-python-headless 4.6.0.66
Uninstalling opencv-python-headless-4.6.0.66:
  Successfully uninstalled opencv-python-headless-4.6.0.66


In [5]:
!pip install opencv-python-headless==4.1.2.30

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.4 MB/s 


In [9]:
# Importing necessary libraries 

import os 
from pathlib import Path 
import xml.etree.ElementTree as ET 
from torch.utils.data import Dataset
from PIL import Image
import torch 
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import numpy as np
import cv2 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

# Downloading the dataset

In [ ]:
 %%bash

wget "https://public.roboflow.com/ds/kQQWZNGxGA?key=65kkvLv3xW" -O data.zip 

unzip -q data.zip

rm data.zip 

ls

# Define Transform 

In [11]:
# def transform
def get_transform(train=True):
    if train:
        return A.Compose([
            A.RandomCrop(height=300, width=300, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightness(p=0.5),
            A.RandomContrast(p=0.5),
            A.RandomGamma(p=0.5),
            A.RandomRotate90(p=0.5),
            A.PadIfNeeded(min_height=512, min_width=512, p=1),
            A.Normalize(p=1, max_pixel_value=255.0, always_apply=True),
            ToTensorV2(p=1.0)
        ] , bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    else:
        return A.Compose([
            A.PadIfNeeded(min_height=300, min_width=512, p=1),
            A.Normalize(p=1, max_pixel_value=255.0, always_apply=True),
            ToTensorV2(p=1.0)
        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [12]:
class MaskDetectionDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = Path(root)
        self.transforms = transforms
        self.image_paths = list(self.root.glob('*.jpg'))
        self.image_paths.sort()
        self.mask_paths = [p.parent / (p.stem + '.xml') for p in self.image_paths]

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Parsing xml to get the labels and the bounding boxes
        annotation = ET.parse(mask_path)
        root = annotation.getroot()
        objects = root.findall('object')
        boxes = []
        labels = []
        for obj in objects:
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(name)

        # Converting the labels and the bounding boxes to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = list(map(lambda x: 1 if x == 'mask' else 0, labels))
        labels = torch.as_tensor(labels)
        image_id = torch.tensor([idx])

        # Other parameters
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        # Creating the tensor for the dataset
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] = iscrowd

        # Applying the transforms
        if self.transforms:
            sample = self.transforms(image = image,
                                     bboxes = target['boxes'],
                                     labels = labels)
            image = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])

        return image, target


    def __len__(self):
        return len(self.image_paths)

In [14]:
# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

ModuleNotFoundError: ignored

In [13]:

train_dataset = MaskDetectionDataset('train', transforms= get_transform(train=True))
train_dl = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1615: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1641: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


NameError: ignored